In [3]:
from Batch import BatchGenerator#生成批次
from bilstm_crf import Model
from utils import*
def run(a):
    import pickle
    import pdb
    import codecs
    import re
    import sys
    import math
    import numpy as np
    import tensorflow as tf
    #import Ipynb_importer
    

    with open('Bosondata.pkl','rb') as inp:
        word2id = pickle.load(inp)
        id2word = pickle.load(inp)
        tag2id = pickle.load(inp)
        id2tag = pickle.load(inp)
        x_train = pickle.load(inp)
        y_train = pickle.load(inp)
        x_test = pickle.load(inp)
        y_test = pickle.load(inp)
        x_valid = pickle.load(inp)
        y_valid = pickle.load(inp)
    print ("train len:",len(x_train))
    print ("test len:",len(x_test))
    print ("word2id len", len(word2id))

    data_train=BatchGenerator(x_train,y_train,shuffle=True)
    data_valid=BatchGenerator(x_valid,y_valid,shuffle=False)
    data_test=BatchGenerator(x_test,y_test,shuffle=False)

    epochs=39
    batch_size=32

    config={}
    config['lr']=0.001
    config['embedding_dim']=100
    config['sen_len']=len(x_train[0])
    config["batch_size"] = batch_size
    config["embedding_size"] = len(word2id)+1
    config["tag_size"] = len(tag2id)
    config["pretrained"]=False

    embedding_pre=[]
    #if len(sys.argv)==2 and sys.argv[1]=='pretrained':
    if a==1:
    #sys.argv[]是用来获取命令行输入的参数的(参数和参数之间空格区分),
    #sys.argv[0]表示代码本身文件路径,所以从参数1开始,表示获取的参数了
        config['pretrained']=True
        word2vec={}
        with codecs.open('vec.txt','r','utf-8') as input_data:
            for line in input_data.readlines():
                word2vec[line.split()[0]]=map(eval,line.split()[1:])
        unknow_pre=[]
        unknow_pre.extend([1]*100)
        embedding_pre.append(unknow_pre)
        for word in word2id:
            if word in word2vec:
                embedding_pre.append(word2vec[word])
            else:
                embedding_pre.append(unknow_pre)
        embedding_pre=np.asarray(embedding_pre)

    #if len(sys.argv)==2 and sys.argv[1]=='test':
    if a==2:
        model=Model(config,embedding_pre,dropout_keep=1)
        with tf.Session as sess:
            sess.run(tf.global_variables_initializer())
            #Session 是 Tensorflow 为了控制,和输出文件的执行的语句. 运行 session.run() 可以获得你要得知的运算结果, 或者是你所要运算的部分
            saver=tf.train.Saver()
            ckpt=tf.train.get_checkpoint_state('model')
            if ckpt is None:
                print('Model not found')
            else:
                path=ckpt.model_checkpoint_path
                saver.restore(sess,path)
                test_input(model,sess,word2id,id2tag,batch_size)

    #elif len(sys.argv)==3:
    if a==3:
        model=Model(config,embedding_pre,dropout_keep=1)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()  
            ckpt = tf.train.get_checkpoint_state('model')
            if ckpt is None:  
                print('Model not found')
            else:    
                path = ckpt.model_checkpoint_path
                saver.restore(sess, path)
                extraction(sys.argv[1],sys.argv[2],model,sess,word2id,id2tag,batch_size)

    if a==0:
        tf.reset_default_graph()
        model = Model(config,embedding_pre,dropout_keep=0.5)
    #经过交叉验证，隐含节点dropout率等于0.5的时候效果最好，原因是0.5的时候dropout随机生成的网络结构最多。
    #dropout也可以被用作一种添加噪声的方法，直接对input进行操作。输入层设为更接近1的数。使得输入变化不会太大（0.8）
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()  
            train(model,sess,saver,epochs,batch_size,data_train,data_test,id2word,id2tag)

In [14]:
def test_input(model,sess,word2id,id2tag,batch_size):
    while True:
        text = input("Enter your input: ");
        text = re.split(u'[，。！？、‘’“”（）]', text) 
        text_id=[]
        for sen in text:
            word_id=[]
            for word in sen:
                if word in word2id:
                    word_id.append(word2id[word])
                else:
                    word_id.append(word2id["unknow"])
            text_id.append(padding(word_id))
        zero_padding=[]
        zero_padding.extend([0]*max_len)
        text_id.extend([zero_padding]*(batch_size-len(text_id)))    
        feed_dict = {model.input_data:text_id}
        pre = sess.run([model.viterbi_sequence], feed_dict)
        entity = get_entity(text,pre[0],id2tag)
        print ('result:')
        for i in entity:
            print (i)

In [ ]:
run(2)

INFO:tensorflow:Restoring parameters from model\model30.ckpt
Enter your input: 奥巴马与2017年12月正式退休
result:
_person_name:奥巴马
_time:2017年12月
Enter your input: 2016年2月19日，党的新闻舆论工作座谈会召开，习近平总书记发表重要讲话。“2•19”重要讲话发表三周年之际，让我们一起重温习近平总书记给新闻舆论战线的“行动指南”。
result:
_time:2016年2月19日
_person_name:习近平
_product_name:2•19
_time:三周年
Enter your input: 海外网2月20日电美国加利福尼亚州、纽约州等16个州日前联合向位于旧金山的加州北部联邦地区法院提起诉讼，指控美国总统特朗普宣布国家紧急状态违反宪法。
result:
_company_name:海外网
_time:2月20日
_location:美国
_location:福尼亚州
_location:纽约州
_location:美国
